In [33]:
import pandas as pd
import re

from maple.types import Action, ActionType
from maple.classification import parse

import warnings
warnings.filterwarnings('ignore')


In [95]:
legislators = [
"Gomez",
"Cronin",
"Gobi",
"Crighton",
"Tarr",
"Stone", 
"Creem",
"Lesser",
"Chandler",
"Boncore",
"Eldridge",
"Lewis",
"Lovely",
"Comerford",
"Keenan",
"Spilka",
"Brady",
"Montigny",
"Moore",
"Pacheco",
"Rodrigues",
"Rush",
"O'Connor",
"Jehlen",
"Rausch",
"Fattman",
"Chang-Diaz",
"DiDomenico",
"Moran",
"Brownsberger",
"Timilty",
"Consalvo",
"Duffy",
"Giannino",
"Kushmerek",
"Pease",
"Philips",
"Scanlon",
"Xiarhos",
"Oliveira",
"Cabral",
"D'Emilia",
"Madaro",
"Ferrante",
"Michlewitz",
"Peisch",
"Puppolo",
"Silvia",
"Ashe",
"Ayers",
"Jones",
"Murray",
"González",
"Doherty",
"Barber",
"Cronin",
"Fiola",
"LeBoeuf",
"Cahill",
"Donahue",
"Garlick",
"Gregoire",
"Linsky",
"Tucker",
"Balser",
"Holmes",
"Mariano",
"Dooley",
"Garballey",
"Harrington",
"Howitt",
"Peake",
"Pignatelli",
"Ultrino",
"Whipps",
"Farley-Bouvier",
"Golden",
"Nguyen",
"Stanley",
"Walsh",
"Whelan",
"Galvin",
"Straus",
"Turco",
"Fernandes",
"Cassidy",
"Kane",
"Hawkins",
"Arciero",
"Wagner",
"Kelcourse",
"McGonagle",
"McKenna",
"O'Day",
"Parisella",
"Roy",
"Zlotnik",
"Lipper-Garabedian",
"Ferguson",
"Gordon",
"Hogan",
"Campbell",
"Ehrlich",
"Sabadosa",
"Malia",
"Mirra",
"Devers",
"Robinson",
"Cusack",
"Day",
"Decker",
"DuBois",
"Finn",
"Keefe",
"Muratore",
"Blais",
"Higgins",
"Boldyga",
"Donato",
"Durant",
"Haddad",
"Mark",
"Schmid",
"Frost",
"Belsito",
"Hinds",
"Finegold",
"Friedman",
"DiZoglio",
"Kennedy",
"Velis",
"Cyr",
"Barrett",
"Collins",
"Feeney",
"Diggs",
"Oakley",
"Ramos",
"Sullivan",
"Vargas",
"Williams",
"Hendricks",
"Minicucci",
"Sena",
"Berthiaume",
"Robertson",
"Rogers",
"Mom",
"Gouveia",
"Moran",
"Lewis"
]


test_string = """16302,Amended (Comerford) by striking out all after the enacting clause and inserting in place thereof the text of n,,uncategorized,,
17190,Amended (DiDomenico) by striking out all after the enacting clause and inserting in place thereof the text of n,,uncategorized,,
20305,Amended (Lewis-Tarr) by striking out all after the enacting clause and inserting in place thereof the text of n,,uncategorized,,
41667,Amended (Tarr),,uncategorized,,"""

ts2 = "Committee of conference (Finegold-Creem-Fattman) appointed,,uncategorized,,"

hash_reg = re.compile('\(#.*\)')

def normalizeLegislatorNames(action: str):
  matches = [l for l in legislators if l in action]
  
  for m in matches: 
    r = '\(.*' + m + '.*\)'
    action = "(###)".join(re.compile(r).split(action))
    print(action)
  action = "(leg_names)".join(hash_reg.split(action))
  return action

normalizeLegislatorNames(ts2)
  

Committee of conference (###) appointed,,uncategorized,,
Committee of conference (###) appointed,,uncategorized,,
Committee of conference (###) appointed,,uncategorized,,


'Committee of conference (leg_names) appointed,,uncategorized,,'

In [96]:
df = pd.read_csv("predictions.csv")

df = df.assign(label=pd.Series(), sublabel=pd.Series())


df['clean_actions'] = df['action'].str.replace(r'[AP]M','xM').str.replace(r'[,.]', '')
df['clean_actions'] = df['clean_actions'].str.replace(r'(?: ?[HS]\d+)+', ' n')
df['clean_actions'] = df['clean_actions'].str.replace(r'\d+ YEAS to \d+ NAYS \(.*$', 'vote')
df['clean_actions'] = df['clean_actions'].str.replace(r'(?:\d+ ?)+ ?(?: ?and \d+)?', 'n ', regex=True)
df['clean_actions'] = df['clean_actions'].str.replace(r'Amendment (?:#n )?\(.*\)', 'Amendment', regex=True)
df['clean_actions'] = df['clean_actions'].str.replace(r'-+ ?see? Roll Call.*$', 'rollcall', regex=True)
df['clean_actions'] = df['clean_actions'].str.replace(r'in the ', 'in ', regex=True)
df['clean_actions'] = df['clean_actions'].str.replace(r'as a ', 'as ', regex=True)
df['clean_actions'] = df['clean_actions'].str.replace(r'of an ', 'of ', regex=True)
df['clean_actions'] = df['clean_actions'].str.replace(r'Reported from the committee on .*', 'Reported from the committee on *', regex=True)
df['clean_actions'] = df['clean_actions'].str.replace(r'The following amendments were considered as one and (\w*)', lambda x: f'The following amendments were considered as one and {x.group(1)}', regex=True)
df['committee'] = df['clean_actions'].str.extract(r'(committee on .*)')
df['clean_actions'] = df['clean_actions'].apply(lambda s: normalizeLegislatorNames(s))

""" 
Accompanied
  new draft
  study order
  by n
  n
"""

drp = df.drop_duplicates(subset='clean_actions')

drp = drp.sort_values(['clean_actions'])

drp.to_csv(path_or_buf='predictions_condensed.csv', columns=['clean_actions','committee', 'prediction','label', 'sublabel'])

Senate concurred
Senate concurred
Committee reported that the matter be placed in Orders of the Day for the next sitting for a second reading
Taken out of the Orders of the Day
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Senate concurred
Committee reported that the matter be placed in Orders of the Day for the next sitting for a